# 🎛️ Interactive Sentiment Demo with Widgets

In [ ]:
import joblib
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

model = joblib.load('../models/best_model.pkl')
tfidf = joblib.load('../models/tfidf_vectorizer.pkl')

In [ ]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return ' '.join(tokens)

In [ ]:
def predict_sentiment(text):
    if not text.strip():
        return "Please enter valid text", 0.0, 0.0, 0.0
    processed = preprocess(text)
    vec = tfidf.transform([processed])
    pred = model.predict(vec)[0]
    proba = model.predict_proba(vec)[0]
    return pred, max(proba), proba[0], proba[1]

In [ ]:
text_input = widgets.Textarea(
    value='Enter your review here...',
    layout=widgets.Layout(width='100%', height='100px')
)
predict_btn = widgets.Button(description='Analyze Sentiment', button_style='primary')
output = widgets.Output()

def on_click(b):
    with output:
        clear_output()
        text = text_input.value
        pred, conf, neg, pos = predict_sentiment(text)
        print(f"Prediction: {pred.upper()} ({conf:.2%})")
        print(f"😊 Positive: {pos:.2%} | 😞 Negative: {neg:.2%}")
        labels = ['Negative', 'Positive']
        plt.bar(labels, [neg, pos], color=['red', 'green'])
        plt.ylim(0, 1)
        plt.title('Sentiment Probabilities')
        plt.show()

predict_btn.on_click(on_click)
display(widgets.VBox([text_input, predict_btn, output]))